In [5]:
#Set up

#install libraries
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import tueplots

import sys
print(sys.executable)

#set source and output paths
source_path = '../../data/'
csv_output_path = '../../data/processed/'
figure_output_path = '../../paper/figures/'

#upload raw query data
df_32_raw = pd.read_csv(f'{source_path}repository_queries/500000_32_homicide-female_DE.csv') 
#upload manually tagged articles with json data
df_tag = pd.read_csv(f'{source_path}processed/manual-tag_all_parsedson.csv')
#filter to only ones with T/F data
df_tag_valid = df_tag.dropna(subset=['woman_murdered'])
#upload keyword data
df_key = pd.read_csv(f'{source_path}manual_tag/femicide_keywords.csv')
#upload top k data
df_50 = pd.read_csv(f'{source_path}processed/7-9_22_24_26-27_31-32_2017-2023_top50.csv')
df_25 = pd.read_csv(f'{source_path}processed/7-14_22-24_26-27_29-32_2017-2023_top25.csv')

/Users/madelinemiller/Desktop/data_literacy/geonews_femicide/source/.venv/bin/python


In [93]:
#filters for raw data
#data filtered to only one entry per NUTS and date between 2017 -2023
df_32_raw['NUTS'] = df_32_raw['NUTS'].fillna('').astype(str)
print(f'df 32 raw  shape: {df_32_raw.shape}')
df_32 = df_32_raw.groupby('id').agg({
    'NUTS': lambda x: ', '.join(sorted(set(code for code in x if code.startswith('DE')))),
    'url': 'first',
    'hostname': 'first',
    'date': 'first',
    'cos_dist': 'first', # these values will all be the same
    'hashed_id': 'first',
    'date_crawled': 'first'
    }).reset_index()
#remove null NUTS
df_32 = df_32[df_32['NUTS'].notna()]
df_32 = df_32[df_32['NUTS']!='']

print(f'df 32 nuts condensed shape: {df_32.shape}')
#Convert the date column to datetime objects
df_32['date'] = pd.to_datetime(df_32['date'])      
# Filter for dates
df_32 = df_32[(df_32['date'].dt.year >= 2017) & (df_32['date'].dt.year <= 2023)]
print(f'df 32 date removed shape: {df_32.shape}')
df_32 = df_32[df_32['cos_dist']<=.225].copy()
print(f'shape of df_32 with threshold: {df_32.shape}')

df 32 raw  shape: (1143913, 13)
df 32 nuts condensed shape: (375215, 8)
df 32 date removed shape: (364570, 8)
shape of df_32 with threshold: (31368, 8)


In [94]:
def removedup_cosdist_source(df):
    #how many total items in dataset have same hostname and cosine distance
    #add column to check article duplicates if the article has the same source and same cosine distance
    df['cos_hostname_duplicate_count'] = df.groupby(['cos_dist', 'hostname'])['cos_dist'].transform('count')

    total_duplicates= df[df['cos_hostname_duplicate_count']>1].shape
    
    # mark retained rows
    df['retained'] = ~df.duplicated(
        subset=['cos_dist', 'hostname'],
        keep='first'
    )
    
    #filter to only rows with duplicates and save csv
    
    df[df['cos_hostname_duplicate_count']>1].to_csv(f'{csv_output_path}duplicate_q32thresh_flagged.csv', index=False)
    
    # keep only retained rows
    df_dupsremoved = df[df['retained']].copy()
    
    print(f'df_dupsremoved: {df_dupsremoved.shape}')
    totaldupsremoved = df.shape[0] - df_dupsremoved.shape[0]
    uniquedups = total_duplicates[0] - totaldupsremoved
    print(f'total duplicates: {total_duplicates[0]}')
    print(f'total_duplicates_removed = {totaldupsremoved}')
    print(f'remaining unique = {uniquedups}')

    return df_dupsremoved

In [95]:
df_32_nodups = removedup_cosdist_source(df_32)
nodup_ids = df_32_nodups['id']

df_tag_nodups = df_tag[df_tag['id'].isin(nodup_ids)]
print(df_tag.shape)
print(df_tag_nodups.shape)

df_dupsremoved: (30195, 10)
total duplicates: 2087
total_duplicates_removed = 1173
remaining unique = 914
(1191, 19)
(604, 19)


In [103]:
def query_relevant_list(df_topk, df_tag_valid,k): 
    # join tagging information to top k
    #left join meand only articles that were part of topk check are retained
    df_query = df_topk.merge(df_tag_valid[['id', 'woman_murdered','URL_works']], on='id', how='left')
    print(f"\nshape topk: {df_topk.shape}")
    print(f"shape topk tagged: {df_query.shape}")
    # Convert all pd.NA to np.nan in the entire dataframe
    df_query = df_query.fillna(np.nan)
    
    # Get list of queries
    queries = [col.removeprefix('cos_rank_') for col in df_query.columns if col.startswith('cos_rank_')]
    
    query_list = []  # List to store all query results
    summary_list = []  # List to store summary per query
    
    # For each query: create records with article id, relevance, cosine distance, and rank
    for query in queries:
        rank_col = f'cos_rank_{query}'
        dist_col = f'cos_dist_{query}'

        # Filter to only rows that have data for this query (non-NaN rank)
        df_query_subset = df_query[df_query[rank_col]<=k].copy()

        # Create updated ranking (only ranks rows where woman_murdered is not null)
        df_query_subset['new_rank'] = np.nan  # Initialize with NaN
        mask = df_query_subset['woman_murdered'].notna()  # Identify rows to rank
        df_query_subset.loc[mask, 'new_rank'] = (
            df_query_subset.loc[mask, dist_col]
                .rank(method='dense', na_option='keep')
                .astype(float)
        )
        
        # Normalized cosine distance
        df_query_subset['normalized_cos_dist'] = (df_query_subset[dist_col] - df_query_subset[dist_col].mean()) / df_query_subset[dist_col].std()

        # Create records for THIS query only (create temporary list)
        current_query_records = []
    
        for _, row in df_query_subset.iterrows():
            record = {
                'query': query,
                'article_id': row['id'],
                'relevance': row['woman_murdered'],
                'cosine_distance': row[dist_col],
                'original_rank': row[rank_col],
                'new_rank': row['new_rank'],
                'url_works': row['URL_works'], 
                'normalized_cos_dist': row['normalized_cos_dist']
            }

            query_list.append(record)
            current_query_records.append(record) 
        
      # Create summary for THIS query using current_query_records
        relevant = sum(1 for r in current_query_records if r['relevance'] == True)
        irrelevant = sum(1 for r in current_query_records if r['relevance'] == False)
        not_checkable = sum(1 for r in current_query_records if pd.isna(r['relevance']))
        
        # Calculate additional summary stats
        original_ranks = [r['original_rank'] for r in current_query_records if not pd.isna(r['original_rank'])]
        new_ranks = [r['new_rank'] for r in current_query_records if not pd.isna(r['new_rank'])]
       
        summary_list.append({
            'query': query,
            'total_articles': len(current_query_records),
            'relevant_count': relevant,
            'irrelevant_count': irrelevant,
            'not_checked_count': not_checkable,
            'max_new_ranks': np.max(new_ranks) if new_ranks else np.nan
        })
    
    # Convert to DataFrames
    df_individual = pd.DataFrame(query_list)
    df_summary = pd.DataFrame(summary_list)
    
    # Print summary
    print(f"Total records created: {len(query_list)}")
    print(f"Number of queries: {len(queries)}")
    
    return queries, df_individual, df_summary

In [132]:
df_50_querylist, df_50_individual, df_50_summary  = query_relevant_list(df_50, df_tag_nodups,50)
df_25_querylist, df_25_individual, df_25_summary = query_relevant_list(df_25, df_tag_nodups,25)


shape topk: (306, 24)
shape topk tagged: (306, 26)
Total records created: 450
Number of queries: 9

shape topk: (263, 40)
shape topk tagged: (263, 42)
Total records created: 425
Number of queries: 17


In [106]:
df_50_individual.to_csv(f'{csv_output_path}test.csv', index=False)
df_25_summary.head(18)

,query,total_articles,relevant_count,irrelevant_count,not_checked_count,max_new_ranks
0,7_police-report-gender-motivated_EN,25,17,0,8,16.0
1,8_police-report-gender-motivated_DE,25,8,2,15,10.0
2,9_woman-girl-killed_EN,25,19,0,6,18.0
3,10_woman-girl-killed_DE,25,14,3,8,13.0
4,11_murder-woman-victim_EN,25,17,4,4,20.0
5,12_murder-woman-victim_DE,25,8,11,6,17.0
6,13_victim-woman-long_EN,25,0,10,15,5.0
7,14_victim-woman-long_DE,25,2,11,12,8.0
8,22_femicide_Femizid_DE,25,4,11,10,15.0
9,23_femicide_Frauenmord_DE,25,9,12,4,19.0


## Accuracy

$$
\text{Accuracy} = \frac{\text{correct}}{\text{total}}
$$



## Precision@k

$$
\text{Precision@k} = \frac{1}{k} \sum_{i=1}^{k} \text{rel}_i
$$

In [134]:
def precisionk(df_summary,k=None,df_raw=None): 
    #filter to rank >= k
    if k != None and df_raw is not None: 
        _, _, df_summary = query_relevant_list(df_raw, df_tag_nodups, k)
    #calculate accuracy
    correct = df_summary['relevant_count']
    total = df_summary['relevant_count'] + df_summary['irrelevant_count']
    
    return correct/total

In [138]:
k = np.min(df_50_summary['max_new_ranks'])
df_50_summary[f'precision{k}'] = precisionk(df_50_summary,k, df_50)
df_50_summary[f'precision27'] = precisionk(df_50_summary,27, df_50)
df_50_summary[f'precision31'] = precisionk(df_50_summary,31, df_50)

df_50_summary.head(18).sort_values(by=[f'precision{k}'], ascending=False)


shape topk: (306, 24)
shape topk tagged: (306, 26)
Total records created: 163
Number of queries: 9

shape topk: (306, 24)
shape topk tagged: (306, 26)
Total records created: 245
Number of queries: 9

shape topk: (306, 24)
shape topk tagged: (306, 26)
Total records created: 279
Number of queries: 9


,query,total_articles,relevant_count,irrelevant_count,not_checked_count,max_new_ranks,precision18.0,precision27,precision31
0,7_police-report-gender-motivated_EN,50,34,0,16,31.0,1.000000,1.000000,1.000000
2,9_woman-girl-killed_EN,50,27,1,22,27.0,1.000000,1.000000,1.000000
8,32_homicide-female_DE,50,45,0,5,41.0,1.000000,1.000000,1.000000
1,8_police-report-gender-motivated_DE,50,14,4,32,18.0,0.750000,0.818182,0.714286
7,31_murder-woman-girl_DE,50,25,11,14,36.0,0.750000,0.631579,0.666667
5,26_femicide_Femicide_EN,50,14,15,21,27.0,0.461538,0.421053,0.428571
6,27_femicide_femicide_EN,50,15,17,18,30.0,0.307692,0.388889,0.476190
3,22_femicide_Femizid_DE,50,9,16,25,25.0,0.272727,0.312500,0.368421
4,24_femicide_Feminizid_DE,50,7,21,22,24.0,0.222222,0.214286,0.187500


## MRR (Mean Reciprocal Rank)
$$
\text{MRR}(q) = \frac{1}{\text{rank of first relevant item in top 50}}
$$


## Normalized Discounted Cumulative Gain (NDCG@50)